In [1]:
%load_ext autoreload
%autoreload 2

# Parameter Estimation
In this notebook I attempt to approach the designation of parameters anew. The main goal is to make clear the source of all parameters, be it from scientific article, or from a computation. The purpose is to make the research more accessible/understandable, as well as ensuring correctness of parameters following a parameter change.

In [2]:
from numpy import exp, sqrt, pi

In [3]:
SHARED_PARAMS = {
    "bbb": [
        ("arteries", "pvs_arteries"),
        ("capillaries", "pvs_capillaries"),
        ("veins", "pvs_veins")
    ],
    "aef": [
        ("pvs_arteries", "ecs"),
        ("pvs_capillaries", "ecs"),
        ("pvs_veins", "ecs")
    ],
    "pvs": ["pvs_arteries", "pvs_capillaries", "pvs_veins"],
    "csf": ["ecs", "pvs_arteries", "pvs_capillaries", "pvs_veins"],
    "blood": ["arteries", "capillaries", "veins"],
    "large_vessels": ["arteries", "veins"],
    "all": ["ecs", "pvs_arteries", "pvs_capillaries", "pvs_veins", "arteries", "capillaries", "veins"]
}

In [4]:
SHARED_PARAMS['large_vessels']

['arteries', 'veins']

In [5]:
def retrieve_aliases(param):
    split = param.split("-")
    return [word for word in split[1:] if word in SHARED_PARAMS]


def unpack_shared_parameters(parameters):
    newparameters = {}
    for param, value in parameters.items():
        aliases = retrieve_aliases(param)
        for alias in aliases:
            replacement = SHARED_PARAMS[alias][0]
            if type(replacement) == str:
                newnames = [param.replace(alias, key) for key in SHARED_PARAMS[alias]]
            elif type(replacement) == tuple and len(replacement) == 2:
                newnames = [param.replace(alias, "-".join(key)) for key in SHARED_PARAMS[alias]]
            else:
                raise IndexError(f"Replacement should be (tuple of) string(s), got {replacement}.")

            for name in newnames:
                split = name.split('-')[1:]
                if not (len(split) > 1 and split[0] == split[1]):
                    newparameters[namee] = value
        
        if len(aliases) == 0:
            newparameters[param] = value
        
    return newparameters


def print_quantities(p):
    for key, value in p.items():
        print(f"{key:<55}: {value}")

In [6]:
from pint import UnitRegistry
from pint import Quantity
# from multirat.parameters import unpack_shared_parameters

ureg = UnitRegistry()

mmHg = ureg.mmHg
Pa = ureg.Pa
m = ureg.m
mm = ureg.mm
um = ureg.um
nm = ureg.nm
s = ureg.s
minute = ureg.min
mL = ureg.mL

In [7]:
base_parameters = {
    # Brain Volume
    "brain_volume": 2450. * mm**3,
    "human_brain_volume": 1.0e6 * mm**3,
    # Pressure boundary parameters
    "pressure_boundary-pvs_arteries": 4.74 * mmHg,
    "pressure_boundary-ecs": 3.74 * mmHg,
    "pressure_boundary-pvs_veins": 3.36 * mmHg,
    "pressure_boundary-arteries": 120. * mmHg,
    "pressure_boundary-veins": 7.0 * mmHg,
    "diffusion_free_inulin": 2.98 * mm**2 / s,
    "tortuosity-all": 1.7,
    "permeability_inulin-bbb": 0.0 * mm / s,  # Does not cross the BBB
    "porosity-ecs": 0.14,
    "vasculature_volume_fraction": 0.0329,
    "vasculature_fraction-arteries": 0.2,  #  (Lee et al. 2001)
    "vasculature_fraction-capillaries": 0.1, # (Lee et al. 2001)
    "vasculature_fraction-veins": 0.7,  # (Lee et al. 2001)
    "pvs_volume_fraction":  0.003,  # (Ballerini, et al. 2020)
    "viscosity-blood": 2.67e-3 * Pa*s,  #  (Guo et al. 2019)
    "viscosity-csf": 7.0e-4 * Pa*s,  #  (Bloomfield, Johnston, Bilston 1998)
    "permeability-ecs": 2.0e-11 * mm**2,
    "resistance-ecs": 4.56 * Pa * s / mm**2,
    # Compute more permeabilities
    "hydraulic_conductivity-ecs-arteries": 9.1e-10 * mm / (Pa * s),
    "hydraulic_conductivity-ecs-capillaries": 1.0e-10 * mm / (Pa * s),
    "hydraulic_conductivity-ecs-veins": 2.0e-11 * mm / (Pa * s),
    "hydraulic_conductivity-capillaries-pvs_capillaries": 2.25e-15 * m**2 *(Pa * s),  # (Asgari 2015 - How astrocyte networks may contribute...)
    "surface_volume_ratio-ecs-arteries": 3. * 1 / mm,   # (Assumed 1/3 of capillary)
    "surface_volume_ratio-ecs-capillaries": 9. * 1 / mm,  # (Smith and Humphrey 2007)
    "surface_volume_ratio-ecs-veins": 3. * 1 / mm,  # (Assumed 1/3 of capillary)
    "flowrate_blood": 700 * mm**3 / s,
    "pressure_drop-arteries-capillaries": 50.0 * mmHg,
    "pressure_drop-capillaries-veins": 10.0 * mmHg,
    "flowrate_csf": 3.33 * mm**3 / s,
    "pressure_drop-pvs_arteries-pvs_capillaries": 1.0 * mmHg,
    "pressure_drop-pvs_capillaries-pvs_veins": 0.25 * mmHg,
    "resistance_network-pvs_arteries": 1.14 * mmHg / (mL / minute),
    "resistance_network-pvs_veins": 1.75e-3 * mmHg / (mL / minute),
    "resistance_network-pvs_capillaries": 32.4 * mmHg / (mL / minute),
    "resistance_network-ecs": 0.57 * mmHg / (mL / minute),
    "resistance_interface-ecs-arteries": 0.57 * mmHg / (mL / minute),
    "resistance_interface-ecs-veins": 0.64 * mmHg / (mL / minute),
    "resistance_interface-ecs-capillaries": 125.31 * mmHg / (mL / minute),
    "thickness_aef_membrane": 1000.0 * nm,  # (Fu, 2010 - Model for BBB permeability to water and small solutes)
    "aef_cleft_distance": 2.5 * nm, # (Fu, 2010) Min value
#     "aef_cleft_distance": 2000.0 * nm,  # Max value
    "diameter-large_vessels": 50.0 * um,
    "diameter-capillaries": 10.0 * um,
    
    # ###### 
    "osmotic_pressure-blood": 20 * mmHg,
    "osmotic_pressure_fraction-csf": 0.2,  # Osm. press. computed as this constat * osmotic_pressure-blood
    "reflection_inulin-bbb": 0.2,
    "reflection_inulin-aef": 0.2,

}
shared_parameters = unpack_shared_parameters(base_parameters)       
print_quantities(base_parameters)

brain_volume                                           : 2450.0 millimeter ** 3
human_brain_volume                                     : 1000000.0 millimeter ** 3
pressure_boundary-pvs_arteries                         : 4.74 millimeter_Hg
pressure_boundary-ecs                                  : 3.74 millimeter_Hg
pressure_boundary-pvs_veins                            : 3.36 millimeter_Hg
pressure_boundary-arteries                             : 120.0 millimeter_Hg
pressure_boundary-veins                                : 7.0 millimeter_Hg
diffusion_free_inulin                                  : 2.98 millimeter ** 2 / second
tortuosity-all                                         : 1.7
permeability_inulin-bbb                                : 0.0 millimeter / second
porosity-ecs                                           : 0.14
vasculature_volume_fraction                            : 0.0329
vasculature_fraction-arteries                          : 0.2
vasculature_fraction-capillaries         

In [8]:
print_quantities(shared_parameters)

brain_volume                                           : 2450.0 millimeter ** 3
human_brain_volume                                     : 1000000.0 millimeter ** 3
pressure_boundary-pvs_arteries                         : 4.74 millimeter_Hg
pressure_boundary-ecs                                  : 3.74 millimeter_Hg
pressure_boundary-pvs_veins                            : 3.36 millimeter_Hg
pressure_boundary-arteries                             : 120.0 millimeter_Hg
pressure_boundary-veins                                : 7.0 millimeter_Hg
diffusion_free_inulin                                  : 2.98 millimeter ** 2 / second
tortuosity-ecs                                         : 1.7
tortuosity-pvs_arteries                                : 1.7
tortuosity-pvs_capillaries                             : 1.7
tortuosity-pvs_veins                                   : 1.7
tortuosity-arteries                                    : 1.7
tortuosity-capillaries                                 : 1.7
tort

In [9]:
def effective_diffusion(D_free, tortuosity):
    return D_free / tortuosity**2


def compute_effective_diffusion(p):
    return {f"diffusion_eff_inulin-{x}": 
            effective_diffusion(
                p[f"diffusion_free_inulin"],
                p[f"tortuosity-{x}"]
            )
        for x in SHARED_PARAMS["all"]
    }


def subnetwork_porosity(network_volume_fraction, subnetwork_fraction):
    return network_volume_fraction * subnetwork_fraction


def compute_vasculature_porosity(p):
    return {
        f"porosity-{x}":
            subnetwork_porosity(
                p['vasculature_volume_fraction'],
                p[f'vasculature_fraction-{x}']
            )
        for x in SHARED_PARAMS["blood"]
    }


def compute_pvs_porosity(p):
    return {
        f"porosity-{x}":
            subnetwork_porosity(
                p['pvs_volume_fraction'],
                p[f'vasculature_fraction-{x.replace("pvs_", "")}']
            )
        for x in SHARED_PARAMS["pvs"]
    }

def compute_porosities(p):
    return {**compute_vasculature_porosity(p), **compute_pvs_porosity(p)}



def compute_permeabilities(p):
    return {
        f"permeability-{x}": 
            permeability(
                p[f'viscosity-{x}'],
                p[f"resistance_network-{x}"],
                p[f"length_area_ratio"]
            )
        for x in [*SHARED_PARAMS["csf"]] #, *SHARED_PARAMS["blood"]]
    }

def compute_length_area_ratio(p):
    kappa = p["permeability-ecs"]
    R = p["resistance-ecs"]
    mu = p["resistance-ecs"]
    return {"length_area_ratio": kappa * R / mu}

def compute_parameters(base):
    """TODO: Add unit converter already here!"""
    out = unpack_shared_parameters(base)
    out = {**compute_effective_diffusion(out), **out}
    out = {**compute_porosities(out), **out}
    out = {**compute_length_area_ratio(out), **out}
    out = {**compute_permeabilities(out), **out}
    return out

In [10]:
def compute_partial_fluid_transfer(brain_volume, resistance, total_transfer):
    new_resistance = 1.0 / (total_transfer * brain_volume) - resistance
    return 1.0 / (new_resistance * brain_volume)


def compute_connected_fluid_transfer(brain_volume, flow_rate, pressure_drop):
    return flow_rate / (pressure_drop * brain_volume)


def compute_convective_fluid_transfer(p):
    """There are several ways to compute these transfer coefficients. Currently
    there seeems to be an issue wrt. the transfer between different compartments
    at the capillary level: If we use the given formula to compute transfer between
    ecs and pericapillary space, why do we need an alternative between capillary and
    venuous? Currently we are using the same formula for all, but I suspect there is
    an error to be found here.
    """
    # FIXME: replace potential error in capillaries/paracapillary resistance.
    ecs_to_blood = {
        f"convective_fluid_transfer-ecs-{x}":
            p[f"hydraulic_conductivity-ecs-{x}"] * p[f"surface_volume_ratio-ecs-{x}"]
        for x in SHARED_PARAMS["blood"]
    }
    ecs_to_pvs = {
        f"convective_fluid_transfer-ecs-{x}":
            1. / (p["human_brain_volume"] * p[f"resistance_interface-ecs-{x.replace('pvs_', '')}"])
        for x in SHARED_PARAMS["pvs"]
    }
    pvs_to_blood = {
        f"convective_fluid_transfer-{x}-pvs_{x}":
            compute_partial_fluid_transfer(
                p["human_brain_volume"],
                p[f"resistance_interface-ecs-{x}"],
                ecs_to_blood[f"convective_fluid_transfer-ecs-{x}"]
            )
        for x in SHARED_PARAMS["blood"]
    }
    connected_blood_transfer = {
        f"convective_fluid_transfer-{i}-{j}":
            compute_connected_fluid_transfer(
                p["human_brain_volume"],
                p[f"flowrate_blood"],
                p[f"pressure_drop-{i}-{j}"]
            )
        for (i, j) in [
            ("arteries", "capillaries"),
            ("capillaries", "veins"),
        ]
    }
    connected_csf_transfer = {
        f"convective_fluid_transfer-{i}-{j}":
            compute_connected_fluid_transfer(
                p["human_brain_volume"],
                p[f"flowrate_csf"],
                p[f"pressure_drop-{i}-{j}"]
            )
        for (i, j) in [
            ("pvs_arteries", "pvs_capillaries"),
            ("pvs_capillaries", "pvs_veins"),
        ]
    }
    convective_transfer = {
        **ecs_to_pvs,
        **pvs_to_blood,
        **connected_blood_transfer,
        **connected_csf_transfer,
    }
    return {
        key: val.to(1/(Pa * s)) for key, val in convective_transfer.items()
    }

p = compute_convective_fluid_transfer(shared_parameters)
print_quantities(p)

convective_fluid_transfer-ecs-pvs_arteries             : 2.1931625024726804e-07 / pascal / second
convective_fluid_transfer-ecs-pvs_capillaries          : 9.976080332051935e-10 / pascal / second
convective_fluid_transfer-ecs-pvs_veins                : 1.953285353764731e-07 / pascal / second
convective_fluid_transfer-arteries-pvs_arteries        : 2.7644107716509763e-09 / pascal / second
convective_fluid_transfer-capillaries-pvs_capillaries  : 9.19849729988111e-09 / pascal / second
convective_fluid_transfer-veins-pvs_veins              : 6.001843614995655e-11 / pascal / second
convective_fluid_transfer-arteries-capillaries         : 1.0500862061839191e-07 / pascal / second
convective_fluid_transfer-capillaries-veins            : 5.250431030919595e-07 / pascal / second
convective_fluid_transfer-pvs_arteries-pvs_capillaries : 2.4977050475660363e-08 / pascal / second
convective_fluid_transfer-pvs_capillaries-pvs_veins    : 9.990820190264145e-08 / pascal / second


In [11]:
print_quantities(compute_parameters(shared_parameters))

permeability-ecs                                       : 2e-11 millimeter ** 2
permeability-pvs_arteries                              : 1.2280701754385965e-14 milliliter * millimeter ** 2 * pascal * second / millimeter_Hg / minute
permeability-pvs_capillaries                           : 4.3209876543209874e-16 milliliter * millimeter ** 2 * pascal * second / millimeter_Hg / minute
permeability-pvs_veins                                 : 8e-12 milliliter * millimeter ** 2 * pascal * second / millimeter_Hg / minute
length_area_ratio                                      : 2e-11 millimeter ** 2
porosity-arteries                                      : 0.00658
porosity-capillaries                                   : 0.00329
porosity-veins                                         : 0.02303
porosity-pvs_arteries                                  : 0.0006000000000000001
porosity-pvs_capillaries                               : 0.00030000000000000003
porosity-pvs_veins                               

### Inulin Transport Parameters

In [ ]:
from multirat.parameters_new import get_base_parameters

In [12]:
def diffusion_pore_cylindrical(D_free, solute_radius, pore_radius):
    beta = solute_radius / pore_radius
    return D_free * (
        1.0 
        - 2.10444 * beta#**6
        + 2.08877 * beta**3
        - 0.094813 * beta**5
        - 1.372 * beta**6
    )

def diffusion_unordered_fibers(D_free, solute_radius, fiber_volume_fraction, fiber_radius):
    return D_free * exp(-sqrt(fiber_volume_fraction)*(1.0 * solute_radius / fiber_radius))

In [13]:
def solute_resistance_layer(layer_thickness, pore_radius, effective_diffusion):
    return layer_thickness / (2.0 * pore_radius * effective_diffusion)

In [14]:
def diffusive_layer_resistances(p, solute):
    pore_layers = ["inner_endothelial_cleft", "endothelial_junction", "outer_endothelial_cleft", 'aef']
    fibrous_layers = ["glycocalyx", "basement_membrane"]
    
    solute_resistances = {}
    for layer in pore_layers:
        D_eff = diffusion_pore_cylindrical(
            p[f"diffusion_free_{solute}"],
            p[f"radius_{solute}"],
            p[f"pore_radius_{layer}"]
        )
        solute_resistances[f"diffusive_resistance_{solute}_{layer}"] = solute_resistance_layer(
            p[f"layer_length_{layer}"], p[f"pore_radius_{layer}"], D_eff
        )
        
    for layer in fibrous_layers:
        membrane_porosity = 1.0 - p[f"fiber_volume_fraction_{layer}"]
        D_eff = diffusion_unordered_fibers(
            p[f"diffusion_free_{solute}"],
            p[f"radius_{solute}"],
            p[f"fiber_volume_fraction_{layer}"],
            p[f"fiber_radius_{layer}"],
        )
        solute_resistances[f"diffusive_resistance_{solute}_{layer}"] = solute_resistance_layer(
            p[f"layer_length_{layer}"], p[f"fiber_radius_{layer}"], D_eff
        )        
    return solute_resistances
    
def diffusive_permeabilities(p, solute):
    R = diffusive_layer_resistances(p, solute)
    bbb_layers = ["glycocalyx","inner_endothelial_cleft", "endothelial_junction",
                  "outer_endothelial_cleft", "basement_membrane"]
    P = {}
    for vi in SHARED_PARAMS["blood"]:
        dvi = p[f"diameter-{vi}"]
        R_bbb = sum([ 1.0 / R[f"diffusive_resistance_{solute}-{layer}"] for layer in bbb_layers])
        R_aef = 1.0 / R[f"diffusive_resistance_{solute}-aef"]
        
        P[f"diffusive_permeability-{vi}-pvs_{vi}"] = R_bbb / (pi * dvi)
        P[f"diffusive_permeability-pvs_{vi}-ecs"] = R_aef / (pi * dvi)
    
    return P

In [15]:
p = diffusive_layer_resistances(
    {**shared_parameters, **base_parameters_permeabilities},
    'inulin')
print_quantities(p)

NameError: name 'base_parameters_permeabilities' is not defined

In [16]:
def diffusion_unordered_fibers(D_free, solute_radius, void_fraction,  fiber_radius):
    return D_free * exp(-sqrt(1.0 - void_fraction)*(1.0 * solute_radius / fiber_radius))

In [17]:
membrane_layers

NameError: name 'membrane_layers' is not defined

In [26]:
membrane_layers = dict(
    glygocalyx = {
        'layer_length': {
            'arteries': 400.0 * nm,
            "capillaries": 250.0 * nm,
            "veins": 100.0 * nm
        },
        "fiber_radius": 6.0 * nm,
        "fiber_volume_fraction": 0.326,
    },
    inner_endothelial_cleft = {
        "layer_length": 350.0 * nm,
        "pore_radius": 9.0 * nm,
    },
    endothelial_cleft_junction = {
        "layer_length": 11.0 * nm,
        'pore_radius': {
            "arteries": 0.5 * nm,
            "capillaries": 2.5 * nm,
            "veins": 10.0 * nm,
        }
    },
    # Outer endothelial cleft computed below
    basement_membrane = {
        'layer_length': {
            'arteries': 80.0 * nm,
            "capillaries": 30.0 * nm,
            "veins": 20.0 * nm,
        },
        "fiber_radius":  2.7 * nm,
        "fiber_volume_fraction": 0.5
    },
    astrocytic_endfeet = {
        "layer_length": 1000.0 * nm,
        "pore_radius": {
            "arteries": 250.0 * nm,
            "capillaries": 10.0 * nm,
            "veins": 250.0 * nm,
        }
    }
)
membrane_layers["outer_endothelial_cleft"] = {
    "layer_length": (
            700.0 * nm 
            - membrane_layers["inner_endothelial_cleft"]["layer_length"]
            - membrane_layers["endothelial_cleft_junction"]["layer_length"]
        ),
    "pore_radius": membrane_layers["endothelial_cleft_junction"]["pore_radius"],
}

def unpack_membrane_layer_vessel(layer):
    unpacked = {}
    for param, value in layer.items():
        for vi in SHARED_PARAMS["blood"]:
            if type(value) == dict:
                unpacked[f"{param}-{vi}"] = value[vi]
            else:
                unpacked[f"{param}-{vi}"] = value
    return unpacked

def unpack_membrane_layers(layers):
    out = {}
    for name, params in layers.items():
        for param, value in unpack_membrane_layer_vessel(params).items():
            out[f"{name}-{param}"] = value
    return out

In [27]:
unpack_membrane_layers(membrane_layers)

{'glygocalyx-layer_length-arteries': 400.0 <Unit('nanometer')>,
 'glygocalyx-layer_length-capillaries': 250.0 <Unit('nanometer')>,
 'glygocalyx-layer_length-veins': 100.0 <Unit('nanometer')>,
 'glygocalyx-fiber_radius-arteries': 6.0 <Unit('nanometer')>,
 'glygocalyx-fiber_radius-capillaries': 6.0 <Unit('nanometer')>,
 'glygocalyx-fiber_radius-veins': 6.0 <Unit('nanometer')>,
 'glygocalyx-fiber_volume_fraction-arteries': 0.326,
 'glygocalyx-fiber_volume_fraction-capillaries': 0.326,
 'glygocalyx-fiber_volume_fraction-veins': 0.326,
 'inner_endothelial_cleft-layer_length-arteries': 350.0 <Unit('nanometer')>,
 'inner_endothelial_cleft-layer_length-capillaries': 350.0 <Unit('nanometer')>,
 'inner_endothelial_cleft-layer_length-veins': 350.0 <Unit('nanometer')>,
 'inner_endothelial_cleft-pore_radius-arteries': 9.0 <Unit('nanometer')>,
 'inner_endothelial_cleft-pore_radius-capillaries': 9.0 <Unit('nanometer')>,
 'inner_endothelial_cleft-pore_radius-veins': 9.0 <Unit('nanometer')>,
 'endothel

In [193]:
{f"{layer}-{param}": val for layername, layer_param in membrane_layers.items() for param, val in unpack_membrane_layer(layer_param).items()
}

{'astrocytic_endfeet-L-arteries': 1000.0 <Unit('nanometer')>,
 'astrocytic_endfeet-L-capillaries': 1000.0 <Unit('nanometer')>,
 'astrocytic_endfeet-L-veins': 1000.0 <Unit('nanometer')>,
 'astrocytic_endfeet-rf-arteries': 6.0 <Unit('nanometer')>,
 'astrocytic_endfeet-rf-capillaries': 6.0 <Unit('nanometer')>,
 'astrocytic_endfeet-rf-veins': 6.0 <Unit('nanometer')>,
 'astrocytic_endfeet-Vf-arteries': 0.326,
 'astrocytic_endfeet-Vf-capillaries': 0.326,
 'astrocytic_endfeet-Vf-veins': 0.326,
 'astrocytic_endfeet-r-arteries': 250.0 <Unit('nanometer')>,
 'astrocytic_endfeet-r-capillaries': 10.0 <Unit('nanometer')>,
 'astrocytic_endfeet-r-veins': 250.0 <Unit('nanometer')>,
 'astrocytic_endfeet-V-arteries': 0.5,
 'astrocytic_endfeet-V-capillaries': 0.5,
 'astrocytic_endfeet-V-veins': 0.5}

{'glygocalyx': {'L': {'arteries': 400.0 <Unit('nanometer')>,
   'capillaries': 250.0 <Unit('nanometer')>,
   'veins': 100.0 <Unit('nanometer')>},
  'rf': 6.0 <Unit('nanometer')>,
  'Vf': 0.326},
 'inner_endothelial_cleft': {'L': 350.0 <Unit('nanometer')>,
  'r': 9.0 <Unit('nanometer')>},
 'endothelial_cleft_junction': {'L': 11.0 <Unit('nanometer')>,
  'r': {'arteries': 0.5 <Unit('nanometer')>,
   'capillaries': 2.5 <Unit('nanometer')>,
   'veins': 10.0 <Unit('nanometer')>}},
 'outer_endothelial_cleft': {'L': 339.0 <Unit('nanometer')>,
  'r': 9.0 <Unit('nanometer')>},
 'basement_membrane': {'L': {'arteries': 80.0 <Unit('nanometer')>,
   'capillaries': 30.0 <Unit('nanometer')>,
   'veins': 20.0 <Unit('nanometer')>},
  'r': 2.7 <Unit('nanometer')>,
  'V': 0.5},
 'astrocytic_endfeet': {'L': 1000.0 <Unit('nanometer')>,
  'r': {'arteries': 250.0 <Unit('nanometer')>,
   'capillaries': 10.0 <Unit('nanometer')>,
   'veins': 250.0 <Unit('nanometer')>}}}

In [164]:
base_parameters_permeabilities = {
    "radius_inulin": 15.2e-7 * mm,  # Source?
    "radius_amyloidbeta": 0.9 * nm,
    "diffusion_free_amyloidbeta": 1.8 * mm**2 / s,
    "layer_length_aef-all": 1000.0 * nm,
    "pore_radius_aef-capillary": 10.0 * nm,
    "pore_radius_aef-large_vessels": 250.0 * nm,
    # Glycocalyx thickness 100-400nm (Fu 2010)
    "layer_length_glycocalyx-veins": 100.0 * nm, 
    "layer_length_glycocalyx-capillaries": 250.0 * nm,
    "layer_length_glycocalyx-arteries": 400.0 * nm,
    # Basement membrane 20 - 80 nm (Fu 2010)
    "layer_length_basement_membrane-capillaries": 30.0 * nm,
    "layer_length_basement_membrane-veins": 20.0 * nm,
    "layer_length_basement_membrane-arteries": 80.0 * nm,
    "fiber_radius_glycocalyx-blood": 6.0 * nm,
    "fiber_radius_basement_membrane": 2.7 * nm,
    "fiber_volume_fraction_glycocalyx-all": 0.326,
    "layer_length_inner_endothelial_cleft-blood": 350.0 * nm,

    "fiber_volume_fraction_basement_membrane": 0.5, 
    "layer_length_endothelial_cleft": 700.0 * nm,
    "layer_length_endothelial_junction": 11 * nm,
    "pore_radius_inner_endothelial_cleft-blood": 9 * nm,
#     "pore_radius_endothelial_junction": 0.5 * nm, # Min value
    "pore_radius_endothelial_junction": 10.0 * nm, # Max Value
    "layer_length_aef": 5000.0 * nm,
    "pore_radius_aef": 5.0 * nm,  # Min Value'
#     "pore_radius_aef": 1000.0 * nm, # Max Value
    
    
}

base_parameters_permeabilities["layer_length_outer_endothelial_cleft"] = (
    base_parameters_permeabilities["layer_length_endothelial_cleft"]
    - base_parameters_permeabilities["layer_length_inner_endothelial_cleft"]
    - base_parameters_permeabilities["layer_length_endothelial_junction"]
)
base_parameters_permeabilities["pore_radius_outer_endothelial_cleft"] = (
    base_parameters_permeabilities["pore_radius_inner_endothelial_cleft"]
)

KeyError: 'layer_length_inner_endothelial_cleft'

In [138]:
p = compute_diffusive_resistances(
    {**shared_parameters, **base_parameters_permeabilities},
    'inulin')
print_quantities(p)

NameError: name 'compute_diffusive_resistances' is not defined

In [128]:
def compute_diffusive_permeability(p, solute):
    R = compute_diffusive_resistances(p, solute)
    L = {}
    for (i, j) in SHARED_PARAMS["aef"]:
        Raef = p[f"diffusive_resistance_{solute}-{i}-{j}"]
        L[f"diffusive_permeability-{i}-{j}"] = diffusive_resistance
    return L

In [ ]:
def compute_diffusive_permeabilities(p, solute):
    layer_resistances = compute_diffusive_perme
    for (i, j) in [*SHARED_PARAMS["bbb"], *SHARED_PARAMS["aef"]]:
        L[f"diffusive_permeability-{i}-{j}"] = 

## Graveyard